Things that didnt work : 

  simple LSTM model

  simple MLP model

  Adding normalization

  balancing data



Sources

Architecture choices - https://arxiv.org/pdf/1809.04356.pdf , 
https://arxiv.org/pdf/1611.06455.pdf



In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import time
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
from sklearn import preprocessing
import functools
from google.colab import drive
from pathlib import Path
drive.mount('/content/drive')

# CONSTANTS
TARGET_PROFIT = 1.1
SEED = 1337
VAL_FRACTION = 0.2
models = ["mlp","fcn","resnet"]
feature_maps = [32,64,128]
batch_sizes = [16,32,64]
lrs = [1e-3,1e-4,1e-5]

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
def normalize_df(df):
  df["ma_gap"] = df.apply(lambda row: [row.ma50[i] - row.ma200[i] for i in range(len(row.ma50))][1:], axis=1)
  df["deriv"] = df["ma200"].apply(lambda x: list(pd.DataFrame(x).diff().iloc[1:,0]))
  df["RSI"] = df["RSI"].apply(lambda x: np.asarray(x[1:])/np.max(x)*2-1)
  df["close"] = df["close"].apply(lambda x: np.asarray(x[1:])/np.max(x)*2-1)
  df["ma50"] = df["ma50"].apply(lambda x: np.asarray(x[1:])/np.max(x)*2-1)
  df["ma200"] = df["ma200"].apply(lambda x: np.asarray(x[1:])/np.max(x)*2-1)
  df["ema200"] = df["ema200"].apply(lambda x: np.asarray(x[1:])/np.max(x)*2-1)
  df["volume"] = df["volume"].apply(lambda x: np.asarray(x[1:])/np.max(x)*2-1)
  return df

In [4]:
def prep_data(df,val=False):

  x_dim = len(df["close"][0])
  y_dim = len(df.columns) - 1

  if val:
    val_df = df.sample(frac=VAL_FRACTION, random_state=SEED)
    train_df = df.drop(val_df.index)
    x_val = np.array([val_df[col] for col in val_df.drop("target",axis=1)], dtype=np.float64).reshape((-1,x_dim,y_dim))
    y_val = np.array(val_df.target, dtype=np.bool)
  else:
    train_df = df
    x_val = None
    y_val = None


  x_train = np.array([train_df[col] for col in train_df.drop("target",axis=1)], dtype=np.float64).reshape((-1,x_dim,y_dim))
  y_train = np.array(train_df.target, dtype=np.bool)

  return x_dim, y_dim, x_train, y_train, x_val, y_val

In [5]:
def build_model(model_type,input_shape,feature_maps,lr):
  if model_type=="resnet":
    return build_resnet(input_shape,feature_maps,lr)
  elif model_type=="fcn":
    return build_fcn(input_shape,feature_maps,lr)
  else:
    return build_mlp(input_shape,feature_maps,lr)

In [6]:
def build_resnet(input_shape,feature_maps,lr):
        input_layer = keras.layers.Input(input_shape)

        # BLOCK 1

        conv_x = keras.layers.Conv1D(filters=feature_maps, kernel_size=8, padding='same')(input_layer)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)

        conv_y = keras.layers.Conv1D(filters=feature_maps, kernel_size=5, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)

        conv_z = keras.layers.Conv1D(filters=feature_maps, kernel_size=3, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        shortcut_y = keras.layers.Conv1D(filters=feature_maps, kernel_size=1, padding='same')(input_layer)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        output_block_1 = keras.layers.add([shortcut_y, conv_z])
        output_block_1 = keras.layers.Activation('relu')(output_block_1)

        # BLOCK 2

        conv_x = keras.layers.Conv1D(filters=feature_maps * 2, kernel_size=8, padding='same')(output_block_1)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)

        conv_y = keras.layers.Conv1D(filters=feature_maps * 2, kernel_size=5, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)

        conv_z = keras.layers.Conv1D(filters=feature_maps * 2, kernel_size=3, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        shortcut_y = keras.layers.Conv1D(filters=feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        output_block_2 = keras.layers.add([shortcut_y, conv_z])
        output_block_2 = keras.layers.Activation('relu')(output_block_2)

        # BLOCK 3

        conv_x = keras.layers.Conv1D(filters=feature_maps * 2, kernel_size=8, padding='same')(output_block_2)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)

        conv_y = keras.layers.Conv1D(filters=feature_maps * 2, kernel_size=5, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)

        conv_z = keras.layers.Conv1D(filters=feature_maps * 2, kernel_size=3, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        shortcut_y = keras.layers.BatchNormalization()(output_block_2)

        output_block_3 = keras.layers.add([shortcut_y, conv_z])
        output_block_3 = keras.layers.Activation('relu')(output_block_3)

        # FINAL

        gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)

        output_layer = keras.layers.Dense(1, activation='sigmoid')(gap_layer)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=lr),
                      metrics=['accuracy'])

        return model

In [7]:
def build_fcn(input_shape,feature_maps,lr):
        input_layer = keras.layers.Input(input_shape)

        conv1 = keras.layers.Conv1D(filters=feature_maps, kernel_size=8, padding='same')(input_layer)
        conv1 = keras.layers.BatchNormalization()(conv1)
        conv1 = keras.layers.Activation(activation='relu')(conv1)

        conv2 = keras.layers.Conv1D(filters=feature_maps*2, kernel_size=5, padding='same')(conv1)
        conv2 = keras.layers.BatchNormalization()(conv2)
        conv2 = keras.layers.Activation('relu')(conv2)

        conv3 = keras.layers.Conv1D(feature_maps, kernel_size=3,padding='same')(conv2)
        conv3 = keras.layers.BatchNormalization()(conv3)
        conv3 = keras.layers.Activation('relu')(conv3)

        gap_layer = keras.layers.GlobalAveragePooling1D()(conv3)

        output_layer = keras.layers.Dense(1, activation='sigmoid')(gap_layer)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=lr),
                      metrics=['accuracy'])

        return model

In [8]:
def build_mlp(input_shape, feature_maps, lr):
  input_layer = keras.layers.Input(input_shape)

  input_layer_flattened = keras.layers.Flatten()(input_layer)
  
  layer_1 = keras.layers.Dropout(0.1)(input_layer_flattened)
  layer_1 = keras.layers.Dense(feature_maps, activation='relu')(layer_1)

  layer_2 = keras.layers.Dropout(0.2)(layer_1)
  layer_2 = keras.layers.Dense(feature_maps, activation='relu')(layer_2)

  layer_3 = keras.layers.Dropout(0.2)(layer_2)
  layer_3 = keras.layers.Dense(feature_maps, activation='relu')(layer_3)

  output_layer = keras.layers.Dropout(0.3)(layer_3)
  output_layer = keras.layers.Dense(1, activation='sigmoid')(output_layer)
  model = keras.models.Model(inputs=input_layer, outputs=output_layer)
  model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=lr),
                metrics=['accuracy'])

  return model

In [9]:
def plot_training_history(history):
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('Training Accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.ylim(-0.1, 1.1)

In [10]:
def test_accuracy(model,x_test,y_test):
    y_pred = model.predict(x_test) > 0.5
    if np.unique(y_pred) == 1:
      return 0
    y_test = y_test.reshape(2030,1)
    return np.count_nonzero(np.logical_and(y_test,y_pred))/len(y_test)

In [11]:
def plot_accuracy(accuracies):
  plt.title('Configuration Accuracy')
  plt.ylabel('Accuracy')
  x = range(len(accuracies))
  plt.bar(x, accuracies.values())
  plt.xticks(x, accuracies.keys())
  plt.ylim(-0.1, 1.1)

In [12]:
def plot_best(accuracies, models):
  plt.title('Model Best Accuracy')
  plt.ylabel('Accuracy')
  x = range(len(models))
  values = [max([accuracies[key] for key in accuracies if model in key]) for model in models]
  plt.bar(x, values)
  plt.xticks(x, models)
  plt.ylim(-0.1, 1.1)

In [13]:
Path("./figs").mkdir(parents=True, exist_ok=True)

data_url = "/content/drive/My Drive/Colab Notebooks/data_set.pkl"
df = pd.read_pickle(data_url).drop("ticker", axis=1)
test_url = "/content/drive/My Drive/Colab Notebooks/test_set.pkl"
test_df = pd.read_pickle(test_url).drop("ticker", axis=1)

In [15]:
df = normalize_df(df)
test_df = normalize_df(test_df)
x_dim, y_dim, x_train, y_train, x_val, y_val = prep_data(df,val=True)
x_test_dim, y_test_dim, x_test, y_test, _, _ = prep_data(test_df)

accuracies = {}

for name in models:
  for maps in feature_maps:
    for lr in lrs:
      model = build_model(name,(x_dim,y_dim),maps,lr)
      model.save_weights('temp.hdf5',)
      for batch_size in batch_sizes:
        model.load_weights('temp.hdf5')
        print(f"Training {name}_FM{maps}_LR{lr}_BS{batch_size}")
        history = model.fit(x_train, y_train, batch_size=batch_size, epochs=100, verbose=0, validation_data=(x_val, y_val))
        model.save(f'{name}_FM{maps}_LR{lr}_BS{batch_size}.hdf5')
        plot_training_history(history)
        plt.savefig(f'figs/training_{name}_FM{maps}_LR{lr}_BS{batch_size}.png')
        plt.close()
        accuracy = test_accuracy(model, x_test, y_test)
        print(f"Accuracy : {accuracy}")
        accuracies[f'{name}_FM{maps}_LR{lr}_BS{batch_size}'] = accuracy 
print("Finished training")
plot_accuracy(accuracies)
plt.savefig(f'figs/accuracies.png')
plt.close()
plot_best(accuracies, models)
plt.savefig(f'figs/best_models.png')
plt.close()

Training mlp_FM32_LR0.001_BS16
Accuracy : 0
Training mlp_FM32_LR0.001_BS32


KeyboardInterrupt: ignored